<a href="https://colab.research.google.com/github/code-cerebrum0/ai/blob/main/TOOLcallingInLangchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import os
os.environ["GOOGLE_API_KEY"] = ""

In [17]:
!pip install -q langchain-google-genai langchain-core requests typing

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [19]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) ->float:
  """This gives exchange rate between base currency and a target currnency"""
  url = f'https://v6.exchangerate-api.com/v6/apikey/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  Given a currency conversion rate this function calculates teh target currency value from a base currency value
  """
  return base_currency_value * conversion_rate

In [20]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
convert.invoke({'base_currency_value': 20, 'conversion_rate': 87.6692})

In [21]:
# tool binding

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])


In [22]:
messages = [HumanMessage("what is the conversion factor beween USD and INR, and based on that can you convert 10 USD to INR.")]

In [23]:
messages

[HumanMessage(content='what is the conversion factor beween USD and INR, and based on that can you convert 10 USD to INR.', additional_kwargs={}, response_metadata={})]

In [25]:
ai_message = llm_with_tools.invoke(messages)

In [29]:
messages.append(ai_message)

In [30]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': 'f5ef7c61-a9bc-4a95-a193-dafcba8d466d',
  'type': 'tool_call'}]

In [31]:
import json

for tool_call in ai_message.tool_calls:
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1=  get_conversion_factor.invoke(tool_call)
    # print(tool_message1)
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']

    meassages.append[tool_message1]

  if tool_call['name'] == 'convert':
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)

    messages.append(tool_message2)



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [32]:
llm_with_tools.invoke(messages)

ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 Unable to submit request because it must include at least one parts field, which describes the prompt input. Learn more: https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/gemini